In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchattacks import PGD, DeepFool
import torch.optim as optim
from torch.utils.data import DataLoader

import Utils
import Models

print(torch.cuda.is_available())

True


In [2]:
def AdversarialAttack(Mynet, X, y, target, Iterations, learningRate, c):
    ListSavedX = []
    for Index in range(len(X)):
        #x = X[Index].unsqueeze(0).clone()
        x = torch.zeros_like(X[Index].unsqueeze(0))
        x = x.requires_grad_(True)
        MyOptimizer = Adam([x], lr=learningRate)
        
        SavedX = x
        PastLoss = 1000 
        for i in range(Iterations):
            goodness_per_label = []
            for label in range(10):
                z = Utils.overlay_y_on_x(x, label)
                goodness = []
                z = Mynet.Layer1(z)
                goodness += [z.pow(2).mean(1)]
                z = Mynet.Layer2(z)
                goodness += [z.pow(2).mean(1)]
                goodness_per_label += [sum(goodness).unsqueeze(1)]
            goodness_per_label = torch.cat(goodness_per_label, 1)

            _, Indexs = torch.topk(goodness_per_label[0], k=2)
            Pred = goodness_per_label[0]
            
            if target != -1:
                Current = Indexs[0]
                if Current == target:
                    Current = Indexs[1]       
                PredLoss = torch.max(Pred[y[Current]] - Pred[target], torch.tensor(-0.001))
            else:
                Current = Indexs[0]
                if Current == y[Index]:
                    Current = Indexs[1]       
                PredLoss = torch.max(Pred[y[Index]] - Pred[Current], torch.tensor(-0.001))

            Loss = PredLoss*c + Utils.L2Distance(x, X[Index])
            #print(i, Loss, PredLoss, Utils.L2Distance(x, X[Index]))
            
            if Loss < PastLoss and PredLoss < 0:
                SavedX = x.clone()
            
            MyOptimizer.zero_grad()
            Loss.backward()
            MyOptimizer.step()
            
            with torch.no_grad():
               x.clamp_(0,1)
            
        ListSavedX.append(SavedX.detach()[0])
    return torch.stack(ListSavedX)

def IterativeNewAttack(Mynet, X, y, target, Iterations, learningRate):
    c = 1000
    AdversarialImages = AdversarialAttack(Mynet, X, y, target, 0, learningRate, c)
    SavedVectors = torch.zeros_like(AdversarialImages)
    
    while c > 0.1:
        print("NET Adversarial with c = ", c)
        AdversarialImages = AdversarialAttack(Mynet, X, y, target, Iterations, learningRate, c)
        AdversarialImages = torch.clamp(AdversarialImages, 0, 1)
        AdversarialLabels = Mynet.predict(AdversarialImages)
        for i in range(len(AdversarialLabels)):
            if y[i].cpu() != AdversarialLabels[i].cpu() and Utils.L2Distance(SavedVectors[i], X[i]) > Utils.L2Distance(AdversarialImages[i], X[i]):
                SavedVectors[i] = AdversarialImages[i]     
        c/=2
    return SavedVectors

In [3]:
torch.manual_seed(1234)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader, test_loader = Utils.MNIST_loaders(test_batch_size=1000)
TestX, TestY = next(iter(test_loader))
TestX, TestY = TestX.cuda(), TestY.cuda()

In [4]:
MyFFModel = Models.Net_MNIST()
Epochs = 5

Utils.FFTrain(MyFFModel, train_loader, Epochs)
Utils.FFEvaluate(MyFFModel, test_loader, device)

Epoch  0
Epoch  1
Epoch  2
Epoch  3
Epoch  4
Accuracy NET: 96.28%


0.9628

In [5]:
MLPModel = Models.MLP_MNIST().to(device)
Epochs = 80
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MLPModel.parameters(), lr=0.001)

Utils.Train(MLPModel, train_loader, device, criterion, optimizer, Epochs)
Utils.Evaluate(MLPModel, test_loader, device)

Epoch 1, Loss: 2.299586772918701
Epoch 2, Loss: 2.218668222427368
Epoch 3, Loss: 2.123408794403076
Epoch 4, Loss: 1.995563268661499
Epoch 5, Loss: 1.8359086513519287
Epoch 6, Loss: 1.6538835763931274
Epoch 7, Loss: 1.4595372676849365
Epoch 8, Loss: 1.2631869316101074
Epoch 9, Loss: 1.0786679983139038
Epoch 10, Loss: 0.9179567098617554
Epoch 11, Loss: 0.7852858901023865
Epoch 12, Loss: 0.6827225685119629
Epoch 13, Loss: 0.604951024055481
Epoch 14, Loss: 0.5479907989501953
Epoch 15, Loss: 0.5048149228096008
Epoch 16, Loss: 0.47189876437187195
Epoch 17, Loss: 0.4467596113681793
Epoch 18, Loss: 0.42621225118637085
Epoch 19, Loss: 0.40978002548217773
Epoch 20, Loss: 0.3964381515979767
Epoch 21, Loss: 0.38457953929901123
Epoch 22, Loss: 0.37440139055252075
Epoch 23, Loss: 0.3652113974094391
Epoch 24, Loss: 0.35650959610939026
Epoch 25, Loss: 0.34852099418640137
Epoch 26, Loss: 0.3410170376300812
Epoch 27, Loss: 0.33372762799263
Epoch 28, Loss: 0.32696428894996643
Epoch 29, Loss: 0.3204578459

0.9551

In [6]:
MyCNNModel = Models.MyCNN_MNIST().to(device)
criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(MyCNNModel.parameters(), lr=0.01)
Epochs = 30

Utils.Train(MyCNNModel, train_loader, device, criterion2, optimizer2, Epochs)
Utils.Evaluate(MyCNNModel, test_loader, device)

Epoch 1, Loss: 2.3057312965393066
Epoch 2, Loss: 2.2919504642486572
Epoch 3, Loss: 2.213318347930908
Epoch 4, Loss: 2.0964293479919434
Epoch 5, Loss: 1.911297082901001
Epoch 6, Loss: 1.686811923980713
Epoch 7, Loss: 1.3928656578063965
Epoch 8, Loss: 1.1166304349899292
Epoch 9, Loss: 0.9137240052223206
Epoch 10, Loss: 0.7623291611671448
Epoch 11, Loss: 0.7152153849601746
Epoch 12, Loss: 0.7631487846374512
Epoch 13, Loss: 0.6541923880577087
Epoch 14, Loss: 0.6647218465805054
Epoch 15, Loss: 0.5794686675071716
Epoch 16, Loss: 0.5234202146530151
Epoch 17, Loss: 0.544903576374054
Epoch 18, Loss: 0.4791417717933655
Epoch 19, Loss: 0.44480186700820923
Epoch 20, Loss: 0.43545427918434143
Epoch 21, Loss: 0.40840643644332886
Epoch 22, Loss: 0.3802521228790283
Epoch 23, Loss: 0.3665936291217804
Epoch 24, Loss: 0.35650312900543213
Epoch 25, Loss: 0.33702054619789124
Epoch 26, Loss: 0.32138127088546753
Epoch 27, Loss: 0.314838707447052
Epoch 28, Loss: 0.30698728561401367
Epoch 29, Loss: 0.292808920

0.9268

In [7]:
LoaderTest = DataLoader(list(zip(TestX, TestY)), batch_size=64, shuffle=False)
X_advFF = IterativeNewAttack(MyFFModel, TestX, TestY,  -1, 1000, 0.01)  

attacks = {
    'PGD': PGD(MLPModel, eps=0.1, alpha=0.02),
    'DeepFool': DeepFool(MLPModel, steps=100, overshoot=0.02)
}

X_advDeep, _ = Utils.generate_adversarial_examples(MLPModel, LoaderTest, attacks['DeepFool'])
X_advDeep = torch.clamp(X_advDeep, 0, 1)

X_advPGD, _ = Utils.generate_adversarial_examples(MLPModel, LoaderTest, attacks['PGD'])
X_advPGD = torch.clamp(X_advPGD, 0, 1)

X_advCW = Utils.IterativeCW(MLPModel, TestX, TestY,  -1, 1000, 0.01)

Utils.DisplayResult(MLPModel, MyFFModel, MyCNNModel, device, TestX, TestY, X_advFF, X_advPGD, X_advCW, X_advDeep)
Utils.WriteResults("ResultsFFAdversrials_MNIST", MLPModel, MyFFModel, MyCNNModel, device, TestX, TestY, X_advFF, X_advPGD, X_advCW, X_advDeep)

NET Adversarial with c =  1000
